In [1]:
# Importar librerías necesarias
from openai import OpenAI
import pandas as pd
import json
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import numpy as np
import random

# Crear una instancia del cliente de OpenAI
client = OpenAI()

In [2]:
# Dataset de pruebas
# Establecer semillas para reproducibilidad
random.seed(42)
np.random.seed(42)

# Número de observaciones
n = 200

# Crear primer DataFrame: Observaciones individuales de aves
df_main_1 = pd.DataFrame({
    "ID_observación": [f"OBS_{i+1:04d}" for i in range(n)],
    "Especie": np.random.choice(
        ["Zorzal", "Chincol", "Tenca", "Picaflor", "Gorrión", "Peuco", "Queltehue", "Diucón"], size=n),
    "Zona": np.random.choice(["Urbana", "Rural", "Suburbana"], size=n),
    "Hora_observación": np.random.choice(
        ["Mañana", "Mediodía", "Tarde", "Noche"], size=n),
    "Duración_observación_min": np.random.normal(loc=30, scale=12, size=n).round(1),
    "Cantidad_individuos": np.random.poisson(lam=4, size=n),
    "Temperatura_C": np.random.normal(loc=19, scale=4.5, size=n).round(1),
    "Humedad_%": np.random.normal(loc=55, scale=15, size=n).round(1),
    "Viento_kmh": np.random.normal(loc=12, scale=5, size=n).round(1),
    "Comportamiento": np.random.choice(
        ["Alimentación", "Canto", "Vuelo", "Reposo", "Interacción", "Defensa de territorio"], size=n),
    "Interacción_humana": np.random.choice(["Sí", "No"], size=n, p=[0.4, 0.6]),
    "Presencia_predador": np.random.choice(["Sí", "No"], size=n, p=[0.3, 0.7]),
    "Fecha_observación": pd.to_datetime(np.random.choice(pd.date_range("2023-01-01", "2023-12-31"), size=n)),
    "Observador": np.random.choice(
        ["Luis", "Camila", "Ignacio", "Valentina", "Marcelo", "Antonia"], size=n),
})

# Segundo DataFrame: Condiciones ecológicas por Zona y Especie
zonas = ["Urbana", "Rural", "Suburbana"]
especies = df_main_1["Especie"].unique()
rows = []

for zona in zonas:
    for especie in especies:
        rows.append({
            "Zona": zona,
            "Especie": especie,
            "Cobertura_vegetal_%": round(np.clip(np.random.normal(30 if zona == "Urbana" else 60, 15), 5, 95), 1),
            "Densidad_humana": np.random.choice(["Alta", "Media", "Baja"]),
            "Nivel_ruido_dB": round(np.random.normal(65 if zona == "Urbana" else 45, 10), 1),
            "Frecuencia_predadores": np.random.choice(["Alta", "Media", "Baja"]),
            "Disponibilidad_nidos": np.random.choice(["Alta", "Media", "Baja"]),
            "Presencia_cuerpos_agua": np.random.choice(["Sí", "No"]),
            "Zona_intervenida": zona == "Urbana" and np.random.rand() < 0.8
        })

df_main_2 = pd.DataFrame(rows)

# Muestras para el sistema
df_sample_1 = df_main_1.head(3).to_string(index=False)
df_sample_2 = df_main_2.head(3).to_string(index=False)

# Previsualizar Data Frames
display(df_main_1.head(5).style.set_table_styles([{
    'selector': 'th',  # Estilo para las celdas de la cabecera
    'props': [('font-size', '10px')]  # Cambiar el tamaño de la fuente a pequeño
}, {
    'selector': 'td',  # Estilo para las celdas de datos
    'props': [('font-size', '10px')]  # Cambiar el tamaño de la fuente a pequeño
}]))

display(df_main_2.head(5).style.set_table_styles([{
    'selector': 'th',  # Estilo para las celdas de la cabecera
    'props': [('font-size', '10px')]  # Cambiar el tamaño de la fuente a pequeño
}, {
    'selector': 'td',  # Estilo para las celdas de datos
    'props': [('font-size', '10px')]  # Cambiar el tamaño de la fuente a pequeño
}]))

,ID_observación,Especie,Zona,Hora_observación,Duración_observación_min,Cantidad_individuos,Temperatura_C,Humedad_%,Viento_kmh,Comportamiento,Interacción_humana,Presencia_predador,Fecha_observación,Observador
0,OBS_0001,Queltehue,Suburbana,Mañana,24.700000,2,21.600000,62.400000,22.300000,Alimentación,No,Sí,2023-07-14 00:00:00,Camila
1,OBS_0002,Picaflor,Suburbana,Mediodía,31.600000,5,26.300000,66.000000,6.700000,Alimentación,No,No,2023-10-07 00:00:00,Camila
2,OBS_0003,Gorrión,Urbana,Mañana,47.300000,5,17.300000,64.900000,12.100000,Defensa de territorio,No,No,2023-10-10 00:00:00,Luis
3,OBS_0004,Queltehue,Urbana,Tarde,12.800000,3,18.100000,72.600000,19.100000,Reposo,Sí,No,2023-02-09 00:00:00,Luis
4,OBS_0005,Tenca,Rural,Tarde,44.000000,1,16.400000,57.700000,11.600000,Vuelo,Sí,No,2023-07-08 00:00:00,Camila


,Zona,Especie,Cobertura_vegetal_%,Densidad_humana,Nivel_ruido_dB,Frecuencia_predadores,Disponibilidad_nidos,Presencia_cuerpos_agua,Zona_intervenida
0,Urbana,Queltehue,21.800000,Baja,73.600000,Media,Alta,Sí,True
1,Urbana,Picaflor,19.600000,Media,65.000000,Baja,Alta,Sí,True
2,Urbana,Gorrión,29.600000,Media,63.500000,Media,Alta,No,True
3,Urbana,Tenca,42.800000,Baja,74.600000,Baja,Baja,No,True
4,Urbana,Diucón,13.700000,Media,56.700000,Baja,Alta,Sí,True


In [3]:
# Configuración GPT
gpt_model = 'gpt-4o' # gpt-3.5-turbo-0125 o gpt-4o

parametros = {
    "max_tokens": 2000,
    "temperature": 0.5,
    "top_p": 0.8,
    "presence_penalty": 0.3,
    "frequency_penalty": 0.5
}

In [4]:
# Crear el Contexto del Sistema

# Formato de respuesta y ejecución
contexto_formato = (
    "Tu salida debe estar en formato JSON, con una única clave 'code'.\n"
    "El valor debe ser una cadena de texto que contenga código Python ejecutable.\n"
    "Usa `df_main_1` y `df_main_2` como referencia a los DataFrames completos disponibles en variables globales.\n"
    "Solo se te muestra un resumen visual (df_sample_1 y df_sample_2).\n"
    "El código debe estar listo para ser ejecutado con `exec()`, sin celdas mágicas ni prints redundantes.\n"
    "Importa explícitamente cualquier librería que vayas a utilizar.\n"
    "Inicializa gráficos con `sns.set_theme()` si incluyes visualizaciones.\n"
    "No generes o modifiques DataFrames externos salvo que se indique expresamente en la pregunta.\n"
)

# Enfoque temático y rol del asistente
contexto_tema = (
    "Eres un asistente experto en análisis de datos con Python.\n"
    "Tu tarea es ayudar al usuario generando código que analice relaciones entre variables\n"
    "y cruce información entre múltiples DataFrames si es necesario.\n"
    "Puedes usar técnicas de agrupación, filtrado, visualización o resumen estadístico.\n"
    "Responde siempre con código funcional y enfocado al análisis solicitado.\n"
)

# Contexto completo combinado
contexto_sistema = contexto_formato + "\n" + contexto_tema

In [15]:
# Crear el Prompt e incorpora la Pregunta
def obtener_respuesta_con_dataframe(pregunta):
    prompt = (
        f"Se muestran muestras de los DataFrames como referencia visual únicamente:\n\n"
        f"df_sample_1:\n{df_sample_1}\n\n"
        f"df_sample_2:\n{df_sample_2}\n\n"
        "Estas muestras son ilustrativas. Los DataFrames completos están disponibles como variables globales `df_main_1` y `df_main_2`.\n\n"
        f"Pregunta del usuario:\n{pregunta}\n\n"
        "Tu tarea es generar **solo código Python** para responder esta pregunta, en un objeto JSON con la estructura exacta:\n\n"
        "`{\"code\": \"<código Python aquí>\"}`\n\n"
        "Instrucciones clave:\n"
        "- El código debe ser completamente **ejecutable** y no debe incluir comentarios fuera del bloque JSON.\n"
        "- Debes usar las variables `df_main_1` o `df_main_2` como referencia al DataFrame completo.\n"
        "- No generes DataFrames desde cero ni cargues archivos a menos que la pregunta lo solicite explícitamente.\n"
        "- Si usas visualizaciones, aplica `sns.set_theme()` y utiliza gráficos de `seaborn` o `matplotlib`.\n"
        "- Usa importaciones estándar como `import pandas as pd`, `import seaborn as sns`, etc., si no están explícitamente presentes.\n\n"
        "No incluyas ninguna explicación textual. Solo retorna el JSON con el bloque de código."
)


    # Consulta API
    try:
        response = client.chat.completions.create(
            model=gpt_model,
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": contexto_sistema},
                {"role": "user", "content": prompt}
            ],
            **parametros  # Usar el diccionario como argumento con **
        )
        
        respuesta_json = response.choices[0].message.content
        codigo_python_dic = json.loads(respuesta_json)

        # Verificar y asegurar el uso de 'df_main' en el código
        codigo_python = codigo_python_dic['code']
        if 'df' not in codigo_python:
            raise ValueError("El código generado no usa 'df_1'.")
        return codigo_python
    except Exception as e:
        print("Error al obtener respuesta de OpenAI:", str(e))
        return None

# Crear widgets interactivos
def crear_widgets(codigo_python):
    widget_codigo = widgets.Textarea(
        value=codigo_python,
        layout=widgets.Layout(width='99%', height='300px'),
        style={'font_family': 'monospace'}
    )

    boton_ejecutar = widgets.Button(description="Ejecutar")
    output_resultado = widgets.Output(
        layout=widgets.Layout(width='99%')
    )

    def ejecutar_codigo(b):
        output_resultado.clear_output()
        codigo_editado = widget_codigo.value
        with output_resultado:
            clear_output(wait=True)
            try:
                exec(codigo_editado, globals())
            except Exception as e:
                print("Error al ejecutar el código:", e)

    boton_ejecutar.on_click(ejecutar_codigo)

    return widget_codigo, boton_ejecutar, output_resultado

In [16]:
# Modulo de consulta y respuesta
pregunta = (
    "¿Puedes analizar la distribución de la duración de observación (`Duración_observación_min`) por zona? "
    "Incluye histogramas separados por zona, con líneas de densidad (`sns.kdeplot`). "
    "Asegúrate de usar `sns.set_theme()` para estilo. "
    "Luego, aplica la prueba de normalidad de Shapiro-Wilk por cada zona. "
    "Entrega una breve interpretación de los resultados según el valor-p. "
    "No muestres valores `NaN`, y trabaja solo con el DataFrame completo `df_main_1`."
)

codigo_python = obtener_respuesta_con_dataframe(pregunta)
if codigo_python:
    widget_codigo, boton_ejecutar, output_resultado = crear_widgets(codigo_python)
    display(widget_codigo, boton_ejecutar, output_resultado)
else:
    print("No se pudo obtener una respuesta de la API de OpenAI.")

Textarea(value="import pandas as pd\nimport seaborn as sns\nimport matplotlib.pyplot as plt\nfrom scipy.stats …

Button(description='Ejecutar', style=ButtonStyle())

Output(layout=Layout(width='99%'))

In [17]:
# Modulo de consulta y respuesta
pregunta = (
    "¿Puedes comparar si hay diferencias significativas en la duración de observación (`Duración_observación_min`) entre zonas `Urbana` y `Rural` usando una prueba t de Student? "
    "Incluye un análisis descriptivo previo con medias y desviaciones estándar por zona. "
    "Realiza la prueba `ttest_ind` de `scipy.stats`, asumiendo varianzas iguales. "
    "Agrega una interpretación simple basada en el valor-p. "
    "Puedes usar visualizaciones como boxplots para mostrar las diferencias. "
    "Aplica `sns.set_theme()` antes de graficar y usa exclusivamente el DataFrame completo `df_main_1`."
)

codigo_python = obtener_respuesta_con_dataframe(pregunta)
if codigo_python:
    widget_codigo, boton_ejecutar, output_resultado = crear_widgets(codigo_python)
    display(widget_codigo, boton_ejecutar, output_resultado)
else:
    print("No se pudo obtener una respuesta de la API de OpenAI.")

Textarea(value='import pandas as pd\nfrom scipy.stats import ttest_ind\nimport seaborn as sns\nimport matplotl…

Button(description='Ejecutar', style=ButtonStyle())

Output(layout=Layout(width='99%'))